# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel');
models = s.models;


In [17]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.00167166, MSLL -3.2398, NLPD -0.3746
Processing energy_data kmeans PoE
PoE: SMSE 0.00204332, MSLL -2.8443, NLPD 0.0208


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [18]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.00138767, MSLL 2.8400, NLPD 5.7052


### 1.1.2 Grid q

In [19]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00156825, MSLL 4.5966, NLPD 7.4617 - q   0.20
TERBCM: SMSE 0.00153641, MSLL 2.2046, NLPD 5.0697 - q   0.40
TERBCM: SMSE 0.00149527, MSLL 1.6313, NLPD 4.4964 - q   0.60
TERBCM: SMSE 0.00144504, MSLL 1.8732, NLPD 4.7383 - q   0.80
TERBCM: SMSE 0.00138767, MSLL 2.8400, NLPD 5.7052 - q   1.00
TERBCM: SMSE 0.00132665, MSLL 4.7455, NLPD 7.6107 - q   1.20
TERBCM: SMSE 0.00126597, MSLL 8.0933, NLPD 10.9585 - q   1.40
TERBCM: SMSE 0.00120888, MSLL 13.8293, NLPD 16.6944 - q   1.60
TERBCM: SMSE 0.00115727, MSLL 23.6710, NLPD 26.5361 - q   1.80
TERBCM: SMSE 0.00111177, MSLL 40.7442, NLPD 43.6093 - q   2.00
TERBCM: SMSE 0.00107215, MSLL 70.7946, NLPD 73.6597 - q   2.20
TERBCM: SMSE 0.00103780, MSLL 124.5082, NLPD 127.3733 - q   2.40
TERBCM: SMSE 0.00100798, MSLL 221.9833, NLPD 224.8484 - q   2.60
TERBCM: SMSE 0.00098200, MSLL 401.4138, NLPD 404.2789 - q   2.80
TERBCM: SMSE 0.00095926, MSLL 736.0671, NLPD 738.9322 - q   3.00


### 1.1.3 Find single q

In [5]:
criterion = 'TERBCM';

In [6]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=0, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -410.93
grad_q_reg =  746.92
ttb =    2.7115e+04
Optimizing q: remain iter - 99, q - 1.5084
grad_q_norm = -393.97
grad_q_reg =  716.61
ttb =    1.9925e+04
Optimizing q: remain iter - 98, q - 1.5164
grad_q_norm = -552.64
grad_q_reg =  1314.8
ttb =    3.6411e+04
Optimizing q: remain iter - 97, q - 1.5277
grad_q_norm = -436.48
grad_q_reg =  457.14
ttb =  6718.0
Optimizing q: remain iter - 96, q - 1.5366
grad_q_norm = -725.32
grad_q_reg =  1339.5
ttb =    2.5592e+04
Optimizing q: remain iter - 95, q - 1.5514
gra

Optimizing q: remain iter - 30, q - 19.5540
grad_q_norm =   -9.7531e+30
grad_q_reg =    2.2680e+60
ttb =    2.6423e+60
Optimizing q: remain iter - 29, q - 20.3540
grad_q_norm =   -8.9454e+30
grad_q_reg =    8.9408e+59
ttb =    1.0477e+63
Optimizing q: remain iter - 28, q - 21.1540
grad_q_norm =   -1.0505e+34
grad_q_reg =    5.6666e+65
ttb =    7.5473e+65
Optimizing q: remain iter - 27, q - 21.9540
grad_q_norm =   -3.0796e+35
grad_q_reg =    6.6424e+68
ttb =    2.2646e+68
Optimizing q: remain iter - 26, q - 22.7540
grad_q_norm =   -2.5891e+29
grad_q_reg =    7.0038e+56
ttb =    3.2952e+71
Optimizing q: remain iter - 25, q - 23.5540
grad_q_norm =   -2.0547e+37
grad_q_reg =    5.5085e+72
ttb =    7.1790e+72
Optimizing q: remain iter - 24, q - 24.3540
grad_q_norm =   -1.4262e+38
grad_q_reg =    2.8089e+74
ttb =    1.7293e+76
Optimizing q: remain iter - 23, q - 25.1540
grad_q_norm =   -1.6684e+36
grad_q_reg =    9.7164e+69
ttb =    3.3675e+78
Optimizing q: remain iter - 22, q - 25.9540
grad

In [7]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [5]:
criterion = 'TERBCM';

In [8]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=0.0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  0.24988
Optimizing qs: remain iter - 99
qs_dist =  0.36819
Optimizing qs: remain iter - 98
qs_dist =  0.25244
Optimizing qs: remain iter - 97
qs_dist =  0.31150
Optimizing qs: remain iter - 96
qs_dist =  0.82423
Optimizing qs: remain iter - 95
qs_dist =  1.0311
Optimizing qs: remain iter - 94
qs_dist =  1.2691
Optimizing qs: remain iter - 93
qs_dist =  0.87230
Optimizing qs: remain iter - 92
qs_dist =  0.89813
Optimizing qs: remain iter - 91
qs_dist =  0.81204
Optimizing qs: remain iter - 90
qs_dist =  1.5894
Optimizing qs: remain iter - 89
qs_dist =  1.0443
Optimizing qs: rem

In [9]:
qs
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   77.1677
    1.4918
   10.8586
   31.1594
    1.6045
    1.7472
   40.6951
   54.0801
   55.6100
   10.0502
    6.9815
    1.2407
    2.1386
   29.1016
    1.2504
    1.4212
    1.3429
    1.4036
    1.2868
   30.9639
   30.4091
    6.5203
   45.9894
   58.8508
   18.0512
    1.7020
   52.3445
    1.3059
    1.2366
    1.8628
   10.6378
    1.5676
   20.2400
   13.5940
    4.7956
    2.0370
   28.8976
    1.4174
    1.5545
   56.1354
    2.6084
   14.2054
    6.1871
    1.7846
    1.4689
    1.6132
    1.3592
    1.8324
    1.5669
   47.6282

TERBCM: SMSE 0.00867647, MSLL 695542599756584852402899478909038491990885284347824668039503064824509468614892482196648606912378480617073489140507803123712.0000, NLPD 695542599756584852402899478909038491990885284347824668039503064824509468614892482196648606912378480617073489140507803123712.0000


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [9]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.00129128, MSLL -4.5474, NLPD -1.6891


### 1.2.2 Grid q

In [10]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00203331, MSLL -4.5420, NLPD -1.6836 - q   0.20
TEGRBCM: SMSE 0.00182699, MSLL -4.5953, NLPD -1.7369 - q   0.40
TEGRBCM: SMSE 0.00158505, MSLL -4.6557, NLPD -1.7974 - q   0.60
TEGRBCM: SMSE 0.00139488, MSLL -4.6700, NLPD -1.8117 - q   0.80
TEGRBCM: SMSE 0.00129128, MSLL -4.5474, NLPD -1.6891 - q   1.00
TEGRBCM: SMSE 0.00127844, MSLL -4.1182, NLPD -1.2599 - q   1.20
TEGRBCM: SMSE 0.00133428, MSLL -3.0635, NLPD -0.2051 - q   1.40
TEGRBCM: SMSE 0.00142008, MSLL -0.7775, NLPD 2.0808 - q   1.60
TEGRBCM: SMSE 0.00149956, MSLL 3.9067, NLPD 6.7650 - q   1.80
TEGRBCM: SMSE 0.00155420, MSLL 13.2681, NLPD 16.1265 - q   2.00
TEGRBCM: SMSE 0.00158313, MSLL 31.8000, NLPD 34.6583 - q   2.20
TEGRBCM: SMSE 0.00159360, MSLL 68.4169, NLPD 71.2752 - q   2.40
TEGRBCM: SMSE 0.00159319, MSLL 140.8951, NLPD 143.7535 - q   2.60
TEGRBCM: SMSE 0.00158711, MSLL 284.8378, NLPD 287.6961 - q   2.80
TEGRBCM: SMSE 0.00157834, MSLL 571.8432, NLPD 574.7016 - q   3.00

### 1.2.3 Find single q

In [10]:
criterion = 'TEGRBCM';

In [11]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=0, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm =  8.6177
grad_q_reg =  58.715
ttb =  667.90
Optimizing q: remain iter - 99, q - 1.4982
grad_q_norm = -3.4305
grad_q_reg =  42.415
ttb =  435.57
Optimizing q: remain iter - 98, q - 1.4989
grad_q_norm = -5.8172
grad_q_reg =  102.56
ttb =  852.24
Optimizing q: remain iter - 97, q - 1.5001
grad_q_norm = -3.9194
grad_q_reg =  45.491
ttb =  493.75
Optimizing q: remain iter - 96, q - 1.5009
grad_q_norm = -10.741
grad_q_reg =  47.121
ttb =  343.87
Optimizing q: remain iter - 95, q - 1.5031
grad_q_norm =  34.180
grad_

grad_q_norm =  83.713
grad_q_reg =  170.37
ttb =  996.75
Optimizing q: remain iter - 22, q - 1.5225
grad_q_norm = -10.565
grad_q_reg =  75.436
ttb =  590.41
Optimizing q: remain iter - 21, q - 1.5247
grad_q_norm = -12.438
grad_q_reg =  116.47
ttb =  928.46
Optimizing q: remain iter - 20, q - 1.5272
grad_q_norm = -13.180
grad_q_reg =  13.379
ttb =  116.50
Optimizing q: remain iter - 19, q - 1.5299
grad_q_norm = -7.0371
grad_q_reg =  15.334
ttb =  142.03
Optimizing q: remain iter - 18, q - 1.5314
grad_q_norm =  22.689
grad_q_reg =  29.145
ttb =  170.36
Optimizing q: remain iter - 17, q - 1.5267
grad_q_norm =  23.140
grad_q_reg =  107.17
ttb =  1013.6
Optimizing q: remain iter - 16, q - 1.5220
grad_q_norm = -6.1586
grad_q_reg =  125.97
ttb =  1008.3
Optimizing q: remain iter - 15, q - 1.5233
grad_q_norm =  9.1137
grad_q_reg =  73.630
ttb =  891.52
Optimizing q: remain iter - 14, q - 1.5214
grad_q_norm =  4.4040
grad_q_reg =  39.531
ttb =  394.28
Optimizing q: remain iter - 13, q - 1.5205


In [12]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [13]:
criterion = 'TEGRBCM';

In [14]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  0.29644
Optimizing qs: remain iter - 99
qs_dist =  0.16004
Optimizing qs: remain iter - 98
qs_dist =  0.12640
Optimizing qs: remain iter - 97
qs_dist =  0.22726
Optimizing qs: remain iter - 96
qs_dist =  0.020871
Optimizing qs: remain iter - 95
qs_dist =  0.84600
Optimizing qs: remain iter - 94
qs_dist =  0.15618
Optimizing qs: remain iter - 93
qs_dist =  0.031457
Optimizing qs: remain iter - 92
qs_dist =  0.050968
Optimizing qs: remain iter - 91
qs_dist =  0.067379
Optimizing qs: remain iter - 90
qs_dist =  0.14171
Optimizing qs: remain iter - 89
qs_dist =  0.042030
Optimizin

In [15]:
qs
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

    2.00141
    1.80976
    1.66381
    2.31366
    1.32835
    1.73756
    1.29406
    1.85963
    1.63445
    0.87347
    0.97901
    1.72504
    1.70797
    1.78590
    1.48277
    1.15667
    1.17175
    1.26740
    1.49231
    1.35855
    1.67301
    1.80214
    1.41111
    1.27849
   18.17951
    1.24552
    1.61257
    1.23442
    1.02481
    1.41582
    1.06688
    1.23552
    1.16138
    2.24957
    1.71476
    1.63965
    1.71523
    1.08720
    0.95871
   11.83213
    1.08334
    1.55624
    1.24908
    0.95856
    1.00854
    1.12923
    1.11412
    1.60704
    2.78469
    1.41901

TEGRBCM: SMSE 2.72782910, MSLL 27914794843693132155977728.0000, NLPD 27914794843693132155977728.0000


## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [23]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.00125745, MSLL 1.5450, NLPD 4.3951


### 1.3.2 Grid q

In [24]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.00152096, MSLL 3.5642, NLPD 6.4143 - q   0.20
TEGPoE: SMSE 0.00146682, MSLL 1.3106, NLPD 4.1606 - q   0.40
TEGPoE: SMSE 0.00140314, MSLL 0.7040, NLPD 3.5541 - q   0.60
TEGPoE: SMSE 0.00133200, MSLL 0.8148, NLPD 3.6649 - q   0.80
TEGPoE: SMSE 0.00125745, MSLL 1.5450, NLPD 4.3951 - q   1.00
TEGPoE: SMSE 0.00118446, MSLL 3.0661, NLPD 5.9162 - q   1.20
TEGPoE: SMSE 0.00111731, MSLL 5.7987, NLPD 8.6487 - q   1.40
TEGPoE: SMSE 0.00105839, MSLL 10.5390, NLPD 13.3891 - q   1.60
TEGPoE: SMSE 0.00100816, MSLL 18.7387, NLPD 21.5888 - q   1.80
TEGPoE: SMSE 0.00096588, MSLL 33.0462, NLPD 35.8963 - q   2.00
TEGPoE: SMSE 0.00093031, MSLL 58.3434, NLPD 61.1934 - q   2.20
TEGPoE: SMSE 0.00090021, MSLL 103.7325, NLPD 106.5826 - q   2.40
TEGPoE: SMSE 0.00087454, MSLL 186.3758, NLPD 189.2259 - q   2.60
TEGPoE: SMSE 0.00085243, MSLL 338.9651, NLPD 341.8151 - q   2.80
TEGPoE: SMSE 0.00083325, MSLL 624.3551, NLPD 627.2052 - q   3.00


### 1.3.3 Find single q

In [25]:
criterion = 'TEGPoE';

In [16]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=0, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -54.239
grad_q_reg =  116.73
ttb =  628.06
Optimizing q: remain iter - 99, q - 1.5011
grad_q_norm = -4.3446
grad_q_reg =  21.567
ttb =  201.07
Optimizing q: remain iter - 98, q - 1.5012
grad_q_norm = -29.848
grad_q_reg =  184.22
ttb =  978.49
Optimizing q: remain iter - 97, q - 1.5018
grad_q_norm =  22.359
grad_q_reg =  108.18
ttb =  1081.2
Optimizing q: remain iter - 96, q - 1.5013
grad_q_norm =  11.869
grad_q_reg =  77.776
ttb =  516.46
Optimizing q: remain iter - 95, q - 1.5011
grad_q_norm = -4.0769
grad_

grad_q_norm =  4.7897
grad_q_reg =  9.4820
ttb =  105.53
Optimizing q: remain iter - 22, q - 1.5076
grad_q_norm = -20.394
grad_q_reg =  51.539
ttb =  581.09
Optimizing q: remain iter - 21, q - 1.5080
grad_q_norm = -13.652
grad_q_reg =  130.69
ttb =  787.72
Optimizing q: remain iter - 20, q - 1.5083
grad_q_norm =  19.065
grad_q_reg =  18.800
ttb =  302.61
Optimizing q: remain iter - 19, q - 1.5079
grad_q_norm = -4.3864
grad_q_reg =  80.154
ttb =  685.75
Optimizing q: remain iter - 18, q - 1.5080
grad_q_norm =  5.0144
grad_q_reg =  7.1444
ttb =  145.71
Optimizing q: remain iter - 17, q - 1.5079
grad_q_norm =  0.78464
grad_q_reg =  6.1444
ttb =  29.873
Optimizing q: remain iter - 16, q - 1.5078
grad_q_norm =  3.1898
grad_q_reg =  35.117
ttb =  323.95
Optimizing q: remain iter - 15, q - 1.5078
grad_q_norm = -5.0294
grad_q_reg =  58.520
ttb =  650.28
Optimizing q: remain iter - 14, q - 1.5079
grad_q_norm = -17.236
grad_q_reg =  92.518
ttb =  639.14
Optimizing q: remain iter - 13, q - 1.5082

In [17]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [18]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  0.014167
Optimizing qs: remain iter - 99
qs_dist =  0.0066318
Optimizing qs: remain iter - 98
qs_dist =  0.018683
Optimizing qs: remain iter - 97
qs_dist =  0.0039270
Optimizing qs: remain iter - 96
qs_dist =  0.012668
Optimizing qs: remain iter - 95
qs_dist =  0.0011460
Optimizing qs: remain iter - 94
qs_dist =  0.020936
Optimizing qs: remain iter - 93
qs_dist =  0.012096
Optimizing qs: remain iter - 92
qs_dist =  0.018839
Optimizing qs: remain iter - 91
qs_dist =  0.056624
Optimizing qs: remain iter - 90
qs_dist =  0.031568
Optimizing qs: remain iter - 89
qs_dist =  0.025572

In [19]:
qs
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.4583
   1.2389
   1.7370
   1.3606
   1.5265
   2.0049
   1.8491
   1.6738
   1.0443
   1.3411
   1.5842
   1.7099
   1.6761
   1.3597
   1.2387
   1.9121
   2.1559
   2.2463
   1.2045
   1.3299
   1.3560
   1.8041
   1.5473
   1.3237
   1.6770
   1.1906
   1.0377
   2.0858
   1.5419
   1.2275
   1.2729
   1.3590
   1.3140
   1.1559
   1.6064
   1.7895
   2.0771
   1.1290
   1.3519
   1.6552
   1.9101
   1.0806
   1.4439
   1.3575
   1.5937
   1.2517
   1.1062
   1.6279
   2.1538
   1.8847

TEGRBCM: SMSE 0.00176099, MSLL 1.9315, NLPD 4.7694
